In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [3]:
facebook_data = pd.read_csv(r"./train.csv")
print(facebook_data.head())
print(facebook_data.describe())
print(facebook_data.shape)

row_id       x       y  accuracy    time    place_id
0       0  0.7941  9.0809        54  470702  8523065625
1       1  5.9567  4.7968        13  186555  1757726713
2       2  8.3078  7.0407        74  322648  1137537235
3       3  7.3665  2.5165        65  704587  6567393236
4       4  4.0961  1.1307        31  472130  7440663949


In [4]:
# 2.基本数据处理
# 2.1 缩小数据范围
facebook_data = facebook_data.query("x>2.0 & x<3.0 & y>2.0 & y<3.0")
# 2.2 选择时间特征
time = pd.to_datetime(facebook_data["time"], unit="s")
time = pd.DatetimeIndex(time)
facebook_data["day"] = time.day
facebook_data["hour"] = time.hour
facebook_data["weekday"] = time.weekday
# 2.3 去掉签到较少的地方
place_count = facebook_data.groupby("place_id").count()
place_count = place_count[place_count["row_id"]>3]
facebook_data = facebook_data[facebook_data["place_id"].isin(place_count.index)]
# 2.4 确定特征值和目标值
x = facebook_data[["x", "y", "accuracy", "day", "hour", "weekday"]]
y = facebook_data["place_id"]
# 2.5 分割数据集
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=22)
print("end")

end


In [16]:
# 3.特征工程--特征预处理(标准化)
# 3.1 实例化一个转换器
transfer = StandardScaler()
# 3.2 调用fit_transform
x_train = transfer.fit_transform(x_train)
x_test = transfer.fit_transform(x_test)
print("end")

end


In [5]:
# 4.机器学习--knn+cv
# 4.1 实例化一个估计器
estimator = KNeighborsClassifier()
# 4.2 调用gridsearchCV
param_grid = {"n_neighbors": [1, 3, 5, 7, 9]}
estimator = GridSearchCV(estimator, param_grid=param_grid, cv=5)
# 4.3 模型训练
estimator.fit(x_train, y_train)
print("end")

end


In [6]:
# 5.模型评估
# 5.1 基本评估方式
score = estimator.score(x_test, y_test)
print("最后预测的准确率为:\n", score)


y_predict = estimator.predict(x_test)
print("最后的预测值为:\n", y_predict)
print("预测值和真实值的对比情况:\n", y_predict == y_test)

# 5.2 使用交叉验证后的评估方式
print("在交叉验证中验证的最好结果:\n", estimator.best_score_)
print("最好的参数模型:\n", estimator.best_estimator_)
print("每次交叉验证后的验证集准确率结果和训练集准确率结果:\n",estimator.cv_results_)

最后预测的准确率为:
 0.050532740250336196
最后的预测值为:
 [5744239220 2886369243 9705652811 ... 9513239029 1824346612 8008821465]
预测值和真实值的对比情况:
 7291186     False
18194106    False
20486694    False
12603090    False
2978665     False
            ...  
27681281    False
15387863    False
17263277    False
19158375    False
10654424    False
Name: place_id, Length: 77336, dtype: bool
在交叉验证中验证的最好结果:
 0.042136849577449245
最好的参数模型:
 KNeighborsClassifier(n_neighbors=1)
每次交叉验证后的验证集准确率结果和训练集准确率结果:
 {'mean_fit_time': array([0.77940254, 0.7447834 , 0.75319128, 0.74901857, 0.74397931]), 'std_fit_time': array([0.0373405 , 0.01491085, 0.02384232, 0.0169693 , 0.01438919]), 'mean_score_time': array([1.82975025, 1.93859925, 2.07061396, 2.11860442, 2.20802932]), 'std_score_time': array([0.04748492, 0.01719793, 0.08460097, 0.01314843, 0.01575088]), 'param_n_neighbors': masked_array(data=[1, 3, 5, 7, 9],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': 